In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import requests
from bs4 import BeautifulSoup 

ID=[]
name=[]
email=[]
designation=[]
img=[]
highest_education=[]
HEC_approved=[]
dept=[]
extension=[]

URL_isb = "http://isb.nu.edu.pk/Faculty/allfaculty/"



In [ ]:
# creating a driver to scrap data from the faculty page using selenium and extracting the faculty member cards
driver1 = webdriver.Chrome()
driver1.get(URL_isb)
faculty_isb = WebDriverWait(driver1, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "our-team"))
)


print(len(faculty_isb))




267


In [7]:
# extracting ID , image , name , email and designation of each member from the faculty page
for f in faculty_isb:
    
    title =f.find_element(By.CLASS_NAME, "title")
    post = f.find_element(By.CLASS_NAME, "post")
    email_addr = f.find_element(By.TAG_NAME, "p")
    image = f.find_element(By.TAG_NAME, "img")
    link = f.find_element(By.TAG_NAME, "a")
    ID.append(int(link.get_attribute("href").split("/")[-1]))
    img.append(image.get_attribute("src"))
    designation.append(post.text)
    name.append(title.text)
    email.append(email_addr.text)
    


print(len(ID))
print(len(img))
print(len(name))
print(len(email))
print(len(designation))

267
267
267
267
267


In [ ]:
# extracting links for each profile and extracting the highest education, extension and if HEC Approved Supervisor 
for f in faculty_isb:
    links = f.find_elements(By.TAG_NAME, "a")

    for link in links:
        profile_link = link.get_attribute('href')
        driver2 = webdriver.Chrome()
        driver2.get(profile_link)
        profile = driver2.find_element(By.CLASS_NAME, "profile-info")
        labels = profile.find_elements(By.TAG_NAME, "label")
        span_elements = profile.find_elements(By.TAG_NAME, "span")

        try:
            degrees = WebDriverWait(driver2, 10).until(
                EC.presence_of_element_located((By.ID, "DegreeName"))
            )

            degree = WebDriverWait(degrees, 30).until(
                EC.presence_of_element_located((By.TAG_NAME, "label"))
            )

            if degree.text:
                print(degree.text)
                highest_education.append(degree.text)

        except:
            print("Not Available")
            highest_education.append("Not Available")

        try:
            if labels[1]:
                HEC_approved.append(True)
        except IndexError:
            HEC_approved.append(False)

        try:
            if span_elements[1]:
                extension.append(int(span_elements[1].text[-3:]))
        except (ValueError, IndexError):
            extension.append(-1)

Ph.D (Computer Science) NUCES, (2010)
Ph.D (Computer Science) National University of Computer & Emerging Sciences (2010)
P.hD (Computer Science) University of Limerick, Ireland (2007)
P.hD (Computer Science) Liverpool John Moore University, UK (2010)
Postdoc (Computer Science) University of Georgia (2019)
PhD (Engineering Science ) Technical University of Vienna (TU WIEN), Vienna, Austria. (2016)
PhD (Computer Science) University of Wales (1996)
Ph.D (Computer Science) University of Innsbruck (2009)
P.hD (Computer Science) University of Cambridge, UK (2011)
P.hD (Computer Science) University of Konstanz, Germany (2011)
Not Available
Ph.D (Computer Science) University of Innsbruck, Austria (2012)
Not Available
P.hD (Computer Science) University of Waterloo, Canada (2013)
Not Available
Ph.D (Computer Science/IT) University of Klagenfurt/University of Genoa, Italy (2019)
P.hD (Computer Science) Kyung Hee University, South Korea (2019)
Not Available
P.hD (Computer Engineering) Kyung Hee Un

In [10]:
# Extracting the dept of each faculty member
driver3 = webdriver.Chrome()


driver3.get(URL_isb)


cs_dept = WebDriverWait(driver3, 10).until(
    EC.presence_of_element_located((By.ID, "facultyCS"))
)


cs_team = WebDriverWait(cs_dept, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, ".//*[contains(@class, 'our-team')]"))
)

for i in range (len(cs_team)):
    dept.append("Computing")



ee_dept = WebDriverWait(driver3, 10).until(
    EC.presence_of_element_located((By.ID, "facultyEE"))
)


ee_team = WebDriverWait(ee_dept, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, ".//*[contains(@class, 'our-team')]"))
)

for i in range (len(ee_team)):
    dept.append("Electrical Engineering")


ms_dept = WebDriverWait(driver3, 10).until(
    EC.presence_of_element_located((By.ID, "facultyMS"))
)

ms_team = WebDriverWait(ms_dept, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, ".//*[contains(@class, 'our-team')]"))
)

for i in range (len(ms_team)):
    dept.append("Management Sciences")


sh_dept = WebDriverWait(driver3, 10).until(
    EC.presence_of_element_located((By.ID, "facultySH"))
)

sh_team = WebDriverWait(sh_dept, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, ".//*[contains(@class, 'our-team')]"))
)

for i in range (len(sh_team)):
    dept.append("Sciences and Humanities")


print(len(dept))

267


In [12]:
print(len(ID), len(name), len(designation), len(HEC_approved), len(highest_education),
      len(email), len(dept), len(extension), len(img))


267 267 267 267 266 267 267 267 267


In [ ]:
# Ensure all lists have the same length
while len(HEC_approved) < len(ID):
    HEC_approved.append(False)  

while len(highest_education) < len(ID):
    highest_education.append("Not Available")  

while len(email) < len(ID):
    email.append("Not Available")

while len(dept) < len(ID):
    dept.append("Unknown")

while len(extension) < len(ID):
    extension.append(-1)  

while len(img) < len(ID):
    img.append("No Image")

In [14]:
# creating the dataframe
isb_df = pd.DataFrame({'ID': ID , 'Name': name , 'Designation': designation , 'HEC Approved PHD Supervisor': HEC_approved ,
                       'Highest Education': highest_education , 'Email' : email , 'Department': dept , 'Extension': extension ,
                       'ImageURL': img })
print(isb_df)

isb_df.to_csv('isb.csv')

       ID            Name                                        Designation  \
0    4553  Waseem Shahzad                               Director & Professor   
1    4551   Hasan Mujtaba              Head, School of Computing & Professor   
2    4400   Hammad Majeed                 HoD (Computer Science) & Professor   
3    5099   Muhammad Asim                   HoD (Cyber Security) & Professor   
4    6478       Ahmad Din  HoD (Artificial Intelligence & Data Science) &...   
..    ...             ...                                                ...   
262  5898      Sana Ilyas                                           Lecturer   
263  5902    Zunera Malik                                           Lecturer   
264  6918  Tayyaba Waseem                                           Lecturer   
265  7007    Maria Mazhar                                           Lecturer   
266  9341    Sadia Nauman                                           Lecturer   

     HEC Approved PHD Supervisor  \
0  